<a href="https://colab.research.google.com/github/LucyKinyua/Week2_MS/blob/main/Copy_of_SQL_Programming_Subqueries_using_Select_Statements.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color="green">*To start working on this notebook, or any other notebook that we will use in the Moringa Data Science Course, we will need to save our own copy of it. We can do this by clicking File > Save a Copy in Drive. We will then be able to make edits to our own copy of this notebook.*</font>

# SQL Programming - Subqueries using Select Statements

## 1.0 Connecting to our Database

In [2]:
# We will first load the sql extension into our environment
%load_ext sql

# Then connect to our in memory sqlite database
# NB: This database will cease to exist as soon as the database connection is closed. 
%sql sqlite://

'Connected: @None'

## 1.1 Loading Data from CSV Files

Datasets used in this notebook:


1.   **World Countries**

> ([Dataset Download Link](https://drive.google.com/a/moringaschool.com/file/d/1XlabneVWhvbmpRUi_Wix2FR5uYAAp-ZY/view?usp=sharing))

2.   **World Cities** 

> ([Dataset Download Link](https://drive.google.com/a/moringaschool.com/file/d/1R5952m-8hH91uiwDK8dcL9sT2D_PEcuw/view?usp=sharing))






In [3]:
# Importing Pandas
import pandas as pd

In [4]:
# Loading our Countries dataset
# 
with open('world_countries.csv','r') as f:
    world_countries = pd.read_csv(f, index_col=0, encoding='utf-8')

%sql PERSIST world_countries;
%sql SELECT * FROM world_countries LIMIT 5;

 * sqlite://
 * sqlite://
Done.


country,population,EU,coastline
Albania,2.9,no,yes
Andorra,0.07,no,no
Austria,8.57,yes,no
Belarus,9.48,no,no
Belgium,11.37,yes,yes


In [5]:
# Loading our cities dataset 
# 
with open('world_cities.csv','r') as f:
    world_cities = pd.read_csv(f, index_col=0, encoding='utf-8')

%sql PERSIST world_cities;
%sql SELECT * FROM world_cities LIMIT 5;

 * sqlite://
 * sqlite://
Done.


city,country,latitude,longitude,temperature
Aalborg,Denmark,57.03,9.92,7.52
Aberdeen,United Kingdom,57.17,-2.08,8.1
Abisko,Sweden,63.35,18.83,0.2
Adana,Turkey,36.99,35.32,18.67
Albacete,Spain,39.0,-1.87,12.62


## 1.2 Subqueries using the SELECT statement

A subquery is a query within another query. The outer query is called as main query and inner query is called a subquery. The subquery must be enclosed with parenthesis and generally executes first, and its output is used to complete the query condition for the main or outer query. We will go through examples of how subqueries can be used to answers different types of questions then later, we will use our learnings to work on the challenges.

In [6]:
# Example 1
# Subqueries can occur in the select list of the containing query
# as shown below;
# Let's find all countries in the Countries table with no city in the Cities table
# 
%%sql
SELECT country
FROM world_countries
WHERE NOT EXISTS (SELECT * FROM world_cities
                  WHERE world_cities.country = world_countries.country) 


 * sqlite://
Done.


country
Cyprus
Iceland
Kosovo
Liechtenstein
Luxembourg


In [8]:
# Example 2
# Find the European countries with cities experiencing temperature > 20?
# 
%%sql
SELECT country
FROM world_countries
WHERE EU = 'yes'
AND EXISTS (SELECT * FROM world_cities
            WHERE world_cities.country = world_countries.country
            AND temperature > 15)

 * sqlite://
Done.


country
Greece
Italy
Portugal
Spain


In [9]:
# Example 3: Method 1
# Find the westernmost city, returning the city and longitude
# 
%%sql
SELECT city, longitude
FROM world_cities C1
WHERE NOT EXISTS (SELECT * FROM world_cities C2
                  WHERE C2.longitude < C1.longitude)

 * sqlite://
Done.


city,longitude
Lisbon,-9.14


In [10]:
# Example 3: Method 2
# Find the westernmost city, returning the city and longitude
# 
%%sql
SELECT city, longitude
FROM world_cities
WHERE longitude = (SELECT min(longitude) FROM world_cities)

 * sqlite://
Done.


city,longitude
Lisbon,-9.14


In [11]:
# Example 4:
# How many cities are in the EU?
# 
%%sql
SELECT count()
FROM world_cities
WHERE country IN (SELECT country FROM world_countries WHERE EU = 'yes')

 * sqlite://
Done.


count()
150


In [12]:
%%sql
SELECT count(city)
FROM world_cities
WHERE country IN (SELECT country FROM world_countries WHERE EU = 'yes')

 * sqlite://
Done.


count(city)
150


### <font color="green">1.2 Challenges</font>

In [35]:
# Challenge 1
# Find number of countries that have a city with latitude > 10.00 (start with country list)
# 
%%sql
SELECT COUNT(country)
FROM world_countries
WHERE EXISTS (SELECT * FROM world_cities
              WHERE world_cities.country = world_countries.country
              AND latitude > 10.00)

 * sqlite://
Done.


COUNT(country)
37


In [22]:
# Challenge 2
# Find three western most cities returning both the cities and their longitude
# 
%%sql
SELECT city, longitude
FROM world_cities
WHERE (SELECT longitude FROM world_cities
       WHERE longitude < 0)
ORDER BY longitude ASC
LIMIT 3;

 * sqlite://
Done.


city,longitude
Lisbon,-9.14
Galway,-9.05
Vigo,-8.73


In [37]:
# Challenge 3
# Find cities with temperature is more than 50% higher than the average; 
# We will return the city, country, and temperature, ordered by descending temperature
# Hint: avg(temperature) * 1.5
# 
%%sql
SELECT city, world_cities.country, temperature
FROM world_cities
WHERE AVG(temperature)*1.5
ORDER BY temperature DESC

 * sqlite://
(sqlite3.OperationalError) misuse of aggregate function AVG()
[SQL: SELECT city, world_cities.country, temperature
FROM world_cities
WHERE AVG(temperature)*1.5
ORDER BY temperature DESC]
(Background on this error at: http://sqlalche.me/e/14/e3q8)


In [16]:
# Challenge 4
# How many cities are not in the EU and without a coastline?
# Hint: "not in"
# 
%%sql
SELECT COUNT(city)
FROM world_cities
WHERE EXISTS (SELECT * FROM world_countries
              WHERE EU = "no"
              AND coastline = "no"
              AND world_cities.country = world_countries.country)

 * sqlite://
Done.


COUNT(city)
16


In [17]:
# Challenge 5
# Find the number of countries with no coastline and a city with longitude < 10
# 
%%sql
SELECT COUNT(country)
FROM world_countries
WHERE coastline = "no"
AND EXISTS (SELECT * FROM world_cities
            WHERE world_cities.country = world_countries.country
            AND longitude < 10)

 * sqlite://
Done.


COUNT(country)
2


In [18]:
# Challenge 6
# Find countries in Countries table with no city in Cities table
# 
%%sql
SELECT country
FROM world_countries
WHERE NOT EXISTS (SELECT * FROM world_cities
                  WHERE world_countries.country = world_cities.country)

 * sqlite://
Done.


country
Cyprus
Iceland
Kosovo
Liechtenstein
Luxembourg


In [19]:
# Challenge 7
# Find all cities in a country whose population is < 2, returning the city and country.
# 
%%sql
SELECT city, country
FROM world_cities
WHERE EXISTS (SELECT * FROM world_countries
             WHERE world_cities.country = world_countries.country
             AND population < 2) 

 * sqlite://
Done.


city,country
Andorra,Andorra
Daugavpils,Latvia
Podgorica,Montenegro
Riga,Latvia
Tallinn,Estonia
Tartu,Estonia


In [20]:
# Challenge 8
# Find all countries with no city having a temperature > 6
# 
%%sql
SELECT country
FROM world_countries
WHERE NOT EXISTS (SELECT * FROM world_cities
                  WHERE world_cities.country = world_countries.country
                  AND temperature > 6) 

 * sqlite://
Done.


country
Cyprus
Estonia
Finland
Iceland
Kosovo
Latvia
Liechtenstein
Luxembourg
Norway
